In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import re

from collections import Counter

import nltk

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

from sklearn import feature_extraction
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

Importing Data

In [11]:
cnn = pd.read_csv (r'C:\Users\zach\cnn_combined.csv')
cnn.head()

,Unnamed: 0,Unnamed: 0.1,title,byline,time,content,date,month_year
0,0,0,Justice Department charges 5 people with helpi...,"By Katelyn Polantz , Kara Scannell and Han...","Updated 4:41 PM ET, Wed March 16, 2022",(cnn) the justice department has charged five...,2022-03-16,2022-03
1,1,1,White House will unveil new data-sharing initi...,"By John Harwood , CNN","Updated 5:09 AM ET, Tue March 15, 2022","(cnn) the biden white house , trying to expa...",2022-03-15,2022-03
2,2,2,Why 'Turning Red' gives me hope,Opinion by Vanessa Hua,"Updated 4:21 PM ET, Wed March 16, 2022",vanessa hua is the author of the forthcoming ...,2022-03-16,2022-03
3,3,3,"5 things to know for March 16: Ukraine, Pandem...","By Alexandra Meeks, CNN","Updated 6:44 AM ET, Wed March 16, 2022",get '5 things' in your inbox if your day doesn...,2022-03-16,2022-03
4,4,4,Senate votes to repeal travel mask mandates in...,"By Ali Zaslav and Ted Barrett , CNN","Updated 7:16 PM ET, Tue March 15, 2022",(cnn) the senate on tuesday passed a resoluti...,2022-03-15,2022-03


In [12]:
fox = pd.read_csv (r'C:\Users\zach\fox_combined.csv')
fox.head()

,Unnamed: 0,Unnamed: 0.1,title,byline,date,content,month_year
0,0,0,COVID loneliness crisis – here's how you can f...,"Nicole Saphier, M.D.",2022-03-15 14:00:00,close video cdc has done a ‘disservice...,2022-03
1,1,1,Republicans go for kill shot on COVID transpor...,Tyler Olson,2022-03-15 14:09:00,close video fox news flash top headlin...,2022-03
2,2,2,10 biggest COVID mistakes – Americans deserve ...,Dr. Marty Makary,2022-03-16 05:00:00,close video dr. makary: covid virus 'b...,2022-03
3,3,3,Lara Trump slams Biden for mask mandates based...,Fox News Staff,2022-03-16 08:23:00,close video lara trump rips biden for ...,2022-03
4,4,4,"Novak Djokovic, Russian players expected to co...",Associated Press,2022-03-16 09:39:00,close video fox news flash top headlin...,2022-03


Setting up article keyword comparison

(CNN) 

In [25]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['the', 'says', 'say', 'a'] # add custom stopwords
stopwords_tokenized = nltk.word_tokenize(' '.join(stopwords))

def stemming(token):
    global stopwords_tokenized
    stemmer = SnowballStemmer("english")
    if (token in stopwords_tokenized):
        return token
    else:
        return stemmer.stem(token)

# a slightly revised process function
def preprocess(text):
    tokens = []
    for sentence in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sentence):
            token = word.lower().replace("'", "")
            if ('covid-19' in token) or ('coronavirus' in token):
                tokens.append('covid')
            else:
                tokens.append(token)
    tokens_filtered = [t for t in tokens if re.search('[a-zA-Z]', t)]
    
    stems = [stemming(t) for t in tokens_filtered]
    return stems
  
articles = cnn.content.tolist()
  
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, max_features=200000, stop_words=stopwords_tokenized,\
                                   strip_accents='unicode', use_idf=True, tokenizer=preprocess, ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(articles)

terms = tfidf_vectorizer.get_feature_names()

# pool top 10 keywords in each news article
cnn_content_words = []
for row in range(tfidf_matrix.shape[0]):
    for i in tfidf_matrix[row].toarray()[0].argsort()[::-1][:10]:
        cnn_content_words.append(terms[i])

(Fox)

In [21]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['the', 'says', 'say', 'a'] # add custom stopwords
stopwords_tokenized = nltk.word_tokenize(' '.join(stopwords))

def stemming(token):
    global stopwords_tokenized
    stemmer = SnowballStemmer("english")
    if (token in stopwords_tokenized):
        return token
    else:
        return stemmer.stem(token)

# a slightly revised process function
def preprocess(text):
    tokens = []
    for sentence in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sentence):
            token = word.lower().replace("'", "")
            if ('covid-19' in token) or ('coronavirus' in token):
                tokens.append('covid')
            else:
                tokens.append(token)
    tokens_filtered = [t for t in tokens if re.search('[a-zA-Z]', t)]
    
    stems = [stemming(t) for t in tokens_filtered]
    return stems
  
articles2 = fox.content.tolist()
  
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, max_features=200000, stop_words=stopwords_tokenized,\
                                   strip_accents='unicode', use_idf=True, tokenizer=preprocess, ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(articles2)

terms = tfidf_vectorizer.get_feature_names()

# pool top 10 keywords in each news article
fox_content_words = []
for row in range(tfidf_matrix.shape[0]):
    for i in tfidf_matrix[row].toarray()[0].argsort()[::-1][:10]:
        fox_content_words.append(terms[i])

Comparing lexical similarities beetween Fox News and CNN

In [22]:
set(fox_content_words).intersection(set(cnn_content_words))

{'tiktok',
 'russian',
 'clinic',
 'loan',
 'mikhail',
 'case',
 'sgt',
 'duchess',
 'canada',
 'pcr test',
 'uk',
 'tucker carlson',
 'fbi',
 'infect omicron',
 'andrew',
 'expert',
 'guy',
 'older',
 'breed',
 'hear',
 'tool',
 'envoy',
 'visit',
 'presidenti',
 'tuesday',
 'state union',
 'fighter',
 'trump',
 'kherson',
 'fed',
 'kent',
 'music',
 'hanna',
 'mental health',
 'school mask',
 'bill',
 'georg',
 'agreement',
 'dmitri',
 'oklahoma',
 'artist',
 'gate',
 'process',
 'congress',
 'predict',
 'contact',
 'reed',
 'garcetti',
 'miss',
 'polic depart',
 'carolina',
 'princ',
 'spike protein',
 'purchas',
 'better',
 'variant',
 'conduct',
 'fire',
 'nation',
 'releas',
 'rescu plan',
 'sunni',
 'order',
 'parent',
 'share',
 'corridor',
 'famili member',
 'florida',
 'graham',
 'leverag',
 'trial',
 'lviv',
 'kit',
 'sorri',
 'cyber',
 'mission',
 'navi',
 'year',
 'donor',
 'becerra',
 'kid',
 'cathol',
 'reelect',
 'bori',
 'neutral',
 'defens minist',
 'patrol',
 'venu',

Comparing lexical differences beetween Fox News and CNN

CNN > Fox News

In [23]:
set(cnn_content_words).difference(set(fox_content_words))

{'year eve',
 'russian aircraft',
 'western offici',
 'backbench',
 'disrupt suppli',
 'disappoint',
 'aviat industri',
 'forest',
 'petit',
 'percentag point',
 'per person',
 'run time',
 'griner russia',
 'accord statement',
 'overnight',
 'speed',
 'inflat expect',
 'fraudul',
 'tycoon',
 'ordinari peopl',
 'gallup',
 'get live',
 'wireless',
 'thermal',
 'store',
 'disclosur',
 'rogozin said',
 'puls',
 'illegitim',
 'like putin',
 'syrian refuge',
 'drought',
 'cortez masto',
 'ballot',
 'busi household',
 'monet',
 'auto',
 'grate',
 'imag show',
 'zuckerberg',
 'truman',
 'manchest',
 'world trade',
 'rail',
 'leg',
 'rice',
 'reliabl sourc',
 'tension rise',
 'n95 kn95',
 'legendari',
 'cancel flight',
 'trevor reed',
 'boss',
 'amanpour',
 'anti-vaccin',
 'prize',
 'stabil',
 'passeng flight',
 'bottl',
 'poland februari',
 'district attorney',
 'adjust',
 'economi continu',
 'talk infrastructur',
 'railway',
 'acquisit',
 'food secur',
 'caption photo',
 'callback',
 'bank s

Fox News > CNN

In [24]:
set(fox_content_words).difference(set(cnn_content_words))

{'studi committe',
 'hand ukrain',
 'efficaci',
 'moham bin',
 'kill children',
 'scienc polit',
 'leader peac',
 'two year',
 'yingst',
 'separ',
 'tom',
 'sting',
 'end',
 'sacramento',
 'avoid',
 'also said',
 'cotton',
 'state-run',
 'twist',
 'half million',
 'transpar',
 'cdc director',
 'culmin',
 'gov chris',
 'anderson',
 'residenti',
 'russia-ukrain war',
 'fox friend',
 'jim jordan',
 'attent',
 'took instagram',
 'middl east',
 'washington post',
 'ukrain poland',
 'refus',
 'holocaust',
 'pierr',
 'gov ned',
 'secretary-gener',
 'southern border',
 'jame comer',
 'take year',
 'photo courtesi',
 'rememb',
 'battlefield',
 'gingrich republican',
 'nathan',
 'swim',
 'nanci',
 'drastic',
 'reach point',
 'thursday even',
 'anoth parent',
 'roll',
 'u.s. energi',
 'marathon',
 'joe biden',
 'perman',
 'converg',
 'photo taken',
 'catherin',
 'ukrainian-polish',
 'hochul said',
 'health emerg',
 'lead caus',
 'shell mariupol',
 'greek',
 'sin',
 'later month',
 'outsid citi',
